<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/2%EC%9B%94%202%EC%9D%BC/model_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 들고 오기

In [32]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

# 데이터 합치기
- product data 중에 이름이 같은 중복데이터가 존재해 이름으로 그룹화하여 index를 다시 만들었음


In [36]:
trans = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/transaction_new.parquet')
customer = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/customer.parquet')
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product_final.parquet')

In [66]:
df = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/df_c.parquet')

In [46]:
trans['created_at']=trans['created_at'].apply(lambda x: x[:19].replace('T',' '))

In [48]:
trans_c = trans[['created_at','customer_id', 'product_id']]

In [50]:
trans_c.sort_values('created_at')

,created_at,customer_id,product_id
1254461,2016-06-30 23:18:44,74089,33314
1201589,2016-07-01 02:53:26,6183,16061
1201590,2016-07-01 02:53:26,6183,7588
1201721,2016-07-01 09:45:51,16228,47650
1254460,2016-07-01 22:32:45,73773,32185
...,...,...,...
985626,2022-07-31 23:58:50,81975,50710
985568,2022-07-31 23:58:53,19551,17773
985766,2022-07-31 23:59:03,22285,27404
985569,2022-07-31 23:59:19,19551,25189


In [70]:
cus=trans_c.groupby('customer_id').count()['created_at'].to_frame().reset_index()
cus['c_idx']= cus.index

In [71]:
cus

,customer_id,created_at,c_idx
0,3,71,0
1,8,8,1
2,9,8,2
3,11,1,3
4,15,9,4
...,...,...,...
50700,99992,17,50700
50701,99995,79,50701
50702,99997,1,50702
50703,99998,24,50703


In [72]:
tran_cus = pd.merge(trans_c, cus[['c_idx','customer_id']], on='customer_id')

In [73]:
tran_cus.head()

,created_at,customer_id,product_id,c_idx
0,2018-07-29 15:22:01,5868,54728,3039
1,2018-07-30 12:40:22,4774,16193,2476
2,2018-09-15 11:51:17,4774,53686,2476
3,2018-11-01 11:23:48,4774,20228,2476
4,2018-12-18 11:20:30,4774,55220,2476


In [83]:
trans_cus_pro = pd.merge(tran_cus, product[['p_idx','product_id']],on='product_id')

In [84]:
trans_cus_pro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253793 entries, 0 to 1253792
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   created_at   1253793 non-null  object
 1   customer_id  1253793 non-null  int32 
 2   product_id   1253793 non-null  uint16
 3   c_idx        1253793 non-null  int64 
 4   p_idx        1253793 non-null  int32 
dtypes: int32(2), int64(1), object(1), uint16(1)
memory usage: 40.7+ MB


In [81]:
# 이걸 진행하니 데이터가 더 증가함 버려버려
# trans_cus_pro['created_at']=trans_cus_pro['created_at'].astype('category')
# trans_cus_pro['c_idx']=trans_cus_pro['c_idx'].astype('int32')

In [85]:
trans_cus_pro['c_idx'].nunique()

50698

In [86]:
trans_cus_pro.to_parquet('model_data.parquet')